In [1]:
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inlin

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [2]:
# Load dataset
df = pd.read_excel('C:/Users/Kornelius/Desktop/Data 2/ecbpr/Warin_Sanger_ECB.xlsx') 
df.head()

,Date,Press,qa
0,1998,"Ladies and gentlemen, in line with our stated ...",We have no indications based on the most recen...
1,1998,"Ladies and gentlemen, as in previous months, t...",The answer to the first question is that it is...
2,1998,"Ladies and gentlemen, the Vice-President and I...",We simply felt that the announcement of a rang...
3,1998,"Ladies and gentlemen, the Vice-President and I...","No, that is not a signal that it is supposed t..."
4,1999,"Ladies and gentlemen, the Vice-President and I...",The main problems have not been technical ones...


In [3]:
#df = df.values.tolist()
# Drop a row by condition
# Print out the first rows of papers
#df = df[df.press]
df = df['qa']

In [4]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(df):
    stop_free = " ".join([i for i in df.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

data = [clean(df).split()for df in df]  

In [5]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [6]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(8371 unique tokens: ['absence', 'accept', 'achieve', 'activity', 'add']...)


In [7]:
texts = data
# less than 10 documents,
id2word = corpora.Dictionary(texts)
id2word.filter_extremes(no_below = 10)

In [8]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA model.

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [9]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [10]:
lda_model5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model20 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model50 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model100 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=100, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [11]:
# Print the Keyword in the 1 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"programme" + 0.008*"thats" + 0.005*"condition" + 0.005*"objective" + '
  '0.005*"actually" + 0.005*"recovery" + 0.005*"weve" + 0.004*"lending" + '
  '0.004*"sense" + 0.004*"bond"')]


In [12]:
# Print the Keyword in the 5 topics
pprint(lda_model5.print_topics())
doc_lda = lda_model5[corpus]

# Print the Keyword in the 10 topics
pprint(lda_model10.print_topics())
doc_lda = lda_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lda_model20.print_topics())
doc_lda = lda_model20[corpus]

# Print the Keyword in the 50 topics
pprint(lda_model50.print_topics())
doc_lda = lda_model50[corpus]

# Print the Keyword in the 100 topics
pprint(lda_model100.print_topics())
doc_lda = lda_model100[corpus]

[(0,
  '0.007*"week" + 0.005*"mr" + 0.005*"forecast" + 0.004*"rather" + 0.004*"day" '
  '+ 0.004*"minister" + 0.004*"conclusion" + 0.004*"earlier" + 0.004*"move" + '
  '0.004*"treaty"'),
 (1,
  '0.021*"bond" + 0.017*"programme" + 0.014*"thats" + 0.011*"asset" + '
  '0.010*"actually" + 0.010*"purchase" + 0.010*"loan" + 0.010*"committee" + '
  '0.009*"sense" + 0.009*"condition"'),
 (2,
  '0.013*"thats" + 0.012*"programme" + 0.009*"objective" + 0.009*"weve" + '
  '0.008*"recovery" + 0.007*"lending" + 0.007*"condition" + 0.007*"basically" '
  '+ 0.006*"especially" + 0.006*"uncertainty"'),
 (3,
  '0.011*"actually" + 0.009*"money" + 0.007*"condition" + 0.007*"balance" + '
  '0.007*"don" + 0.007*"ab" + 0.007*"sheet" + 0.006*"consolidation" + '
  '0.006*"funding" + 0.006*"operation"'),
 (4,
  '0.006*"deliver" + 0.005*"observed" + 0.005*"staff" + 0.005*"currency" + '
  '0.004*"quarter" + 0.004*"credible" + 0.004*"functioning" + '
  '0.004*"extremely" + 0.004*"creation" + 0.004*"observing"')]
[(

[(44,
  '0.000*"labour" + 0.000*"bias" + 0.000*"credibility" + 0.000*"try" + '
  '0.000*"productivity" + 0.000*"cost" + 0.000*"eye" + 0.000*"asset" + '
  '0.000*"extremely" + 0.000*"universe"'),
 (49,
  '0.000*"uncertainty" + 0.000*"letter" + 0.000*"mr" + 0.000*"reserve" + '
  '0.000*"procedure" + 0.000*"pact" + 0.000*"rest" + 0.000*"slow" + '
  '0.000*"lack" + 0.000*"minister"'),
 (89,
  '0.000*"value" + 0.000*"shock" + 0.000*"conclusion" + 0.000*"specific" + '
  '0.000*"indicator" + 0.000*"enough" + 0.000*"week" + 0.000*"payment" + '
  '0.000*"day" + 0.000*"reference"'),
 (72,
  '0.000*"forecast" + 0.000*"potential" + 0.000*"pleased" + 0.000*"closely" + '
  '0.000*"eurogroup" + 0.000*"surprised" + 0.000*"reference" + 0.000*"become" '
  '+ 0.000*"often" + 0.000*"currency"'),
 (32,
  '0.000*"size" + 0.000*"conclusion" + 0.000*"uncertainty" + '
  '0.000*"intervention" + 0.000*"precisely" + 0.000*"currency" + 0.000*"day" + '
  '0.000*"indicator" + 0.000*"reserve" + 0.000*"upside"'),
 (37

# Evaluation
Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity5: ', lda_model5.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity10: ', lda_model10.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity20: ', lda_model20.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity50: ', lda_model50.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity100: ', lda_model100.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100: ', coherence_lda)


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence ScoreUMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100UMass: ', coherence_lda)


Perplexity:  -7.4913506857198

Perplexity5:  -7.28014685079901

Perplexity10:  -7.320147780709629

Perplexity20:  -7.3502609818078

Perplexity50:  -7.460561003029341

Perplexity100:  -9.6822365105836

Coherence Score:  0.5141230588077896

Coherence Score5:  0.4737840304861427

Coherence Score10:  0.41563586865754

Coherence Score20:  0.34899172781550314

Coherence Score50:  0.30583183484967286

Coherence Score100:  0.2939294253412934

Coherence ScoreUMass:  -0.6436320867333867

Coherence Score5UMass:  -0.8230028093186907

Coherence Score10UMass:  -1.3116029317527271

Coherence Score20UMass:  -0.9591360641831088

Coherence Score50UMass:  -1.1101171513458765

Coherence Score100UMass:  -1.240727765244423
